In [3]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_ad4580c044b745b7a5ebdface9d435fe(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_1cf9d5012ae7df1c9487c5cf4accf2254fb13cd9','domain': {'id': 'a6138a977e1a4c2ab44ddc1e0e8019de'},
            'password': 'R4}9)R4yxq4yc=OF'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_ad4580c044b745b7a5ebdface9d435fe('cbf', 'rev 1.csv'))
df_data_1.head()

,TYPE,Sum of LATITUDE,Sum of LONGITUDE,Sum of Count
0,Parking Enforcement,42.23,-71.12,3
1,Parking Enforcement,42.23,-71.13,7
2,Parking Enforcement,42.24,-71.11,18
3,Parking Enforcement,42.24,-71.12,30
4,Parking Enforcement,42.24,-71.13,27


In [4]:
import requests # library to handle requests
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from pandas.io.json import json_normalize

foursquare_client_id = "ZHJOGWSCPK5CWSL3HYGEPPHHELO4KLQL0BPFMF5FWTHBUZ3A"
foursquare_client_secret = "G40TRZURRK53W3M02Z53V30LC31EEEULO4L4UPPHTBK05F1R"
query = ''

address = ''
if address == '':
    latitude = 42.2957
    longitude = -71.0617
else :
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

radius = 1000
LIMIT = 30
VERSION = 20171006
url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}".format(
    foursquare_client_id, foursquare_client_secret, latitude, longitude, VERSION, query, radius, LIMIT)

results = requests.get(url).json()

venues = results["response"]["venues"]  # assign relevant part of JSON to venues
if len(venues) == 0:
    reply = 'Sorry, I couldn\'t find any doctors near you.'
else:

    dataframe = json_normalize(venues)  # tranform venues into a dataframe

    filtered_columns = ['name', 'url', 'categories', 'verified'] + [col for col in dataframe.columns if
                                                                    col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.ix[:, filtered_columns]


    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row["categories"]
        except:
            categories_list = row["venue.categories"]

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]["name"].encode('ascii', errors='ignore')


    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split(".")[-1] for column in dataframe_filtered.columns]
    reply = 'Here is what I found:\n';

    for venue in dataframe_filtered.name:
        if len(reply) > 0:
            reply = reply + '\n'
            reply = reply + '* ' + venue
            print(reply)


Here is what I found:

* Eddy's Auto Repair Irving
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
* Flora's Beauty Supply
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
* Flora's Beauty Supply
* black star books
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
* Flora's Beauty Supply
* black star books
* O'Brien's Market & Deli
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
* Flora's Beauty Supply
* black star books
* O'Brien's Market & Deli
* Gadget Repair King Wireless
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
* Flora's Beauty Supply
* black star books
* O'Brien's Market & Deli
* Gadget Repair King Wireless
* Gibson Playground
Here is what I found:

* Eddy's Auto Repair Irving
* Keaney funeral home
* Flora's Beauty Supply
* black star books
* O'Brien's Market & Deli
* Gadget Repair

In [5]:
dataframe_filtered['categories']=="Bar" or "Dessert Shop" or 

0                                  Gas Station
1                                 Funeral Home
2                               Cosmetics Shop
3                                    Bookstore
4                            Convenience Store
5                            Electronics Store
6                                   Playground
7                                         None
8                           Salon / Barbershop
9                                   Eye Doctor
10                           Convenience Store
11                           Elementary School
12                              Baseball Field
13                                Dessert Shop
14                                Liquor Store
15    Residential Building (Apartment / Condo)
16                           Elementary School
17                                Tech Startup
18                           Mobile Phone Shop
19                                        Park
20                             Other Nightlife
21           

In [6]:
categories_list 

NameError: name 'categories_list' is not defined